# Introduction

This notebook is for an experiment for sample size.  
https://www.kaggle.com/atsushiiwasaki/how-many-samples-do-i-want

Because it is light somewhat, I choose
* DISTILBERT

Hyper-parameters / Optimizer / Scheduler or any settings for them are not optimized.  

This notebook is based on  
* Training Part: https://www.kaggle.com/atsushiiwasaki/commonlit-bert-stratified-k-fold-baseline-train  
* Inference Part: https://www.kaggle.com/atsushiiwasaki/commonlit-bert-stratified-k-fold-baseline-infer

# Libraries

In [ ]:
import os
import random
import gc
from pprint import pprint
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(style='darkgrid')

from sklearn.model_selection import StratifiedKFold

%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AutoModel
from transformers import AutoTokenizer
from transformers import AutoConfig

# Configuration

In [ ]:
DEBUG = False

TRAIN = '../input/commonlitreadabilityprize/train.csv'
TEST = '../input/commonlitreadabilityprize/test.csv'

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: ', device.type)

SEED = 28
seed_everything(SEED)

cfg ={}

In [ ]:
# BERT
BERT = 'bert-base-uncased'

# Distilbert
DISTILBERT = 'distilbert-base-uncased'

# Roberta
ROBERTA = 'roberta-base'



cfg ={}

ARCH_PATH = DISTILBERT

cfg['train'] = {'n_folds': 5}

# Data

## Stratify on target distribution

In [ ]:
def get_bin_stratified(df, n_bins=10, n_splits=5):
    df['bin'] = pd.cut(df.target, n_bins, labels=[i for i in range(n_bins)])
    
    df['fold'] = np.nan

    skf = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)
    gen_skf = skf.split(df.id, y=df.bin)

    for fold, (idx_train, idx_val) in enumerate(gen_skf):
        df.loc[idx_val, 'fold'] = fold

    df['fold'] = df['fold'].astype('int8')

In [ ]:
df = pd.read_csv(TRAIN)
get_bin_stratified(df)

plt.figure(figsize=(12, 6))
for fold in range(cfg['train']['n_folds']):
    sns.histplot(data=df.loc[df.fold==fold], x='target', bins=10, hue='fold', label=f'fold{fold}')
    
plt.title('Target Distribution for Each Fold')
plt.legend()
plt.show()

## Tokenizer

In [ ]:
cfg['tokenizer'] ={'name': ARCH_PATH, 
                   'max_length': 210}

tokenizer = AutoTokenizer.from_pretrained(cfg['tokenizer']['name'])

In [ ]:
if DEBUG:
    text = df.loc[SEED, 'excerpt']
    print('Text Length ', len(text.split(' ')))
    print()
    
    text_tokenized = tokenizer.encode_plus(
                        text,
                        add_special_tokens=True,
                        padding='max_length',
                        max_length=cfg['tokenizer']['max_length'], 
                        truncation=True
                        )
    
    for key, value in text_tokenized.items():
        print(key, type(value))
        print(value)
        print()

## Dataset

In [ ]:
class CommonLitDataset(Dataset):
    
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.df.loc[index, 'excerpt']
        inputs = self.tokenizer.encode_plus(
            text,                                 
            add_special_tokens=True,
            padding='max_length',
            max_length=self.max_len,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
        if cfg['tokenizer']['name']=='bert-base-uncased':
            token_type_ids = inputs['token_type_ids'] 
        else:
            token_type_ids = 1.
        
        target = self.df.loc[index, ['target']]
        
        return {
            'ids': torch.LongTensor(ids),
            'mask': torch.LongTensor(mask),
             'token_type_ids': torch.tensor(token_type_ids)
            },{
            'target': torch.Tensor(target)
        }

## Dataloader

In [ ]:
cfg['dl_train'] = {
    'batch_size': 8 if device.type=='cpu' else 32, 
    'shuffle': True, 
    'num_workers': os.cpu_count(), 
    'pin_memory': True
}

cfg['dl_val'] = {
    'batch_size': 8 if device.type=='cpu' else 64, 
    'shuffle': False, 
    'num_workers': os.cpu_count(), 
    'pin_memory': True
}

# Model

In [ ]:
cfg['model'] = {'name': ARCH_PATH}

In [ ]:
class CommonLitBERT(nn.Module):
    
    def __init__(self, name, dropout=True):
        super(CommonLitBERT, self).__init__()
        self.bert = AutoModel.from_pretrained(name)
        self.name = name
        
        if name == BERT:
            self.in_features = self.bert.pooler.dense.out_features
        elif name == DISTILBERT:
            self.in_features = self.bert.transformer.layer[5].output_layer_norm.normalized_shape[0]
        elif name == ROBERTA:
            self.in_features = self.bert.pooler.dense.out_features
        else:
            self.in_features = 768
        
        self.fc = nn.Linear(self.in_features, 1)
        self.dense = nn.Linear(self.in_features, self.in_features)
        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, ids, mask, token_type_ids):
        if self.name == BERT:
            last_hidden_state, output = self.bert(ids,
                                                  attention_mask=mask,
                                                  token_type_ids=token_type_ids,
                                                  return_dict=False)
        elif self.name == DISTILBERT:
            last_hidden_state = self.bert(ids, 
                                           attention_mask=mask, 
                                           return_dict=False)
            first_token_tensor = last_hidden_state[0][:, 0]
            output = self.dense(first_token_tensor)
            output = self.activation(output)
            
        elif self.name == ROBERTA:
            last_hidden_state, output = self.bert(ids,
                                                  attention_mask=mask,
#                                                   token_type_ids=token_type_ids,
                                                  return_dict=False)
        output = self.dropout(output)
        output = self.fc(output)
        return output

# Criterion

In [ ]:
def CommonLitMetric(y_pred, y_gt):
    assert y_pred.size() == y_gt.size()
    
    metric = nn.MSELoss()
    metric = torch.sqrt(metric(y_pred, y_gt))
    return metric

# Optimizer

In [ ]:
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

from transformers import get_cosine_schedule_with_warmup
from transformers import AdamW

cfg['optim'] = {'lr': 5e-5}
cfg['scheduler'] = {'num_warmup_steps': 2, 
                    'num_training_steps': 10, 
#                     'num_cycles': 1,
                   }

# Training / Inference

In [ ]:
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

In [ ]:
cfg['train'] ={
    'n_folds': 5,
    'n_epochs': 100
}

In [ ]:
class StoreLoss:
    
    def __init__(self, fold):
        self.loss_train_mean = []
        self.loss_train_std = []
        self.loss_val_mean = []
        self.loss_val_std = []
        
        self.fold = fold
        
    def get_loss(self, loss_train, loss_val):
        self.loss_train_mean.append(loss_train[0])
        self.loss_train_std.append(loss_train[1])
        self.loss_val_mean.append(loss_val[0])
        self.loss_val_std.append(loss_val[1])
        
    def plot_loss(self):
        
        def get_ax(ax, loss_train, loss_val, title='mean'):
            ax.plot(loss_train, marker='o', label='train')
            ax.plot(loss_val, marker='x', label='val')
            ax.set_xlabel('Epoch')
            ax.set_ylabel(f'RMSE ({title})')
            ax.set_title(f'RMSE({title}) vs Epoch at fold {self.fold}')
            ax.legend()
            return ax
        
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
        
        ax[0] = get_ax(ax[0], self.loss_train_mean, self.loss_val_mean, title='mean')
        ax[1] = get_ax(ax[1], self.loss_train_std, self.loss_val_std, title='std')
        
        
        fig.show()

In [ ]:
def train_fn(model, dl, criterion, optim, scheduler):
    scaler = GradScaler()
    
    loss_train = []
    loss_total = 0
    
    model.train()
    model.to(device)
    
    progress_bar = tqdm(dl, desc='train')
    
    for i, data in enumerate(progress_bar):
        optim.zero_grad()
        
        inputs = {key: value.to(device) for key, value in data[0].items()}
        targets = data[1]['target'].to(device)
        
        with autocast():
            outputs = model(**inputs)
            loss = criterion(outputs, targets)
        
        scaler.scale(loss).backward()
        
        loss_train.append(loss.item())
        loss_total += loss.item()
        
        progress_bar.set_postfix({'RMSE(batch)': loss.item(), 
                                  'RMSE(ave)': loss_total / (i+1), 
                                  'lr': optim.param_groups[0]['lr']})
        
        scaler.step(optim)
        
        scaler.update()
    
    return np.mean(loss_train), np.std(loss_train)

def val_fn(model, dl):
    scaler = GradScaler()
    
    loss_val = []
    loss_total = 0
    
    model.eval()
    model.to(device)
    
    progress_bar = tqdm(dl, desc='val')
    
    with torch.no_grad():
        for i, data in enumerate(progress_bar):
            inputs = {key: value.to(device) for key, value in data[0].items()}
            targets = data[1]['target'].to(device)
            
            with autocast():
                outputs = model(**inputs)
                loss = CommonLitMetric(outputs, targets)
            
            loss_val.append(loss.item())
            loss_total += loss.item()
            
            progress_bar.set_postfix({'RMSE(batch)': loss.item(), 'RMSE(ave)': loss_total / (i+1)})
    
    loss_val_2 = np.array(loss_val)**2 * cfg['dl_val']['batch_size'] / len(dl.dataset)
    print('RMSE for validation set overall: ', np.sqrt(loss_val_2.sum()))
    
    return np.sqrt(loss_val_2.sum()), np.std(loss_val)

In [ ]:
def run_one_epoch(model, train_dl, val_dl, criterion, optim, scheduler):
    inputs_train = {
        'model': model, 
        'dl': train_dl, 
        'criterion': criterion, 
        'optim': optim, 
        'scheduler': scheduler
    }

    inputs_val = {'model': model, 
                  'dl': val_dl}

    loss_train = train_fn(**inputs_train)
    loss_val = val_fn(**inputs_val)
    
    return loss_train, loss_val

In [ ]:
def get_dls_for_n_fold(df, fold, tokenizer):
    train_df = df.loc[df.fold!=fold].reset_index(drop=True)
    val_df = df.loc[df.fold==fold].reset_index(drop=True)
    
    train_ds = CommonLitDataset(
        train_df, 
        tokenizer=tokenizer, 
        max_len=cfg['tokenizer']['max_length']
    )
    
    val_ds = CommonLitDataset(
        val_df, 
        tokenizer=tokenizer, 
        max_len=cfg['tokenizer']['max_length']
    )
    
    train_dl = DataLoader(train_ds, **cfg['dl_train'])
    val_dl = DataLoader(val_ds, **cfg['dl_val'])
    
    return train_dl, val_dl

In [ ]:
# https://qiita.com/ku_a_i/items/ba33c9ce3449da23b503

class EarlyStopping:
    
    def __init__(self, patience=2, seq=False):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.stop = False
        
    def __call__(self, loss, model, optim, cfg, path):
        if self.best_score is None:
            self.best_score = loss
            self.save_checkpoint(model, optim, cfg, path)
        elif loss < self.best_score:
            print(f'Loss decreased {self.best_score} -> {loss}.')
            self.best_score = loss
            self.counter = 0
            self.save_checkpoint(model, optim, cfg, path)
        else:
            self.counter += 1
            if self.counter > self.patience: self.stop = True
                
    def save_checkpoint(self, model, optim, cfg, path):
        save_list = {'model': model.state_dict(), 
#                      'optim': optim.state_dict(), 
                     'cfg': cfg}
        SAVE_PATH = path
        torch.save(save_list, SAVE_PATH)

# Run

In [ ]:
pprint(cfg)

In [ ]:
def main(n_samples):
        
    df = pd.read_csv(TRAIN)
    get_bin_stratified(df, n_splits=cfg['train']['n_folds'])
    
    df = df.sample(n_samples, random_state=SEED).reset_index(drop=True)

    tokenizer = AutoTokenizer.from_pretrained(cfg['tokenizer']['name'])
    
    for fold in range(cfg['train']['n_folds']):
        print('Fold:', fold)
        store = StoreLoss(fold=fold)
        es = EarlyStopping()

        train_dl, val_dl = get_dls_for_n_fold(df, fold, tokenizer)

        model = CommonLitBERT(name=cfg['model']['name'])
        criterion = CommonLitMetric
        optim = AdamW(model.parameters(), **cfg['optim'])
        scheduler = get_cosine_schedule_with_warmup(optim, **cfg['scheduler'])
        
        if optim.param_groups[0]['lr']==0:
            optim.step()
            scheduler.step()

        inputs = {'model': model,
                  'train_dl': train_dl,
                  'val_dl': val_dl,
                  'criterion': criterion,
                  'optim': optim,
                  'scheduler': scheduler}

        for epoch in range(cfg['train']['n_epochs']):
            loss_train, loss_val = run_one_epoch(**inputs)
            
            store.get_loss(loss_train, loss_val)
            
            es(loss_val[0], model, optim, cfg, path=f'BERT_fold{fold}_n_samples{n_samples}.tar')
            if es.stop:
                print('Early Stop !')
                print()
                break

            scheduler.step()
            
        store.plot_loss()
        
        del model, optim
        gc.collect()

In [ ]:
%%time
df = pd.read_csv(TRAIN)
n_samples_set = [1000, 1500, 2000, 2500, len(df)]

for n_samples in n_samples_set:
    print('Number of Samples: ', n_samples)
    main(n_samples=n_samples)